In [1]:
import json
import sys
sys.path.append("../")

split = "test"
rewrite_method = "slimplmqr"
dataset = "hotpot-qa"
version="v0715"
search_engine="bing"

node_file = f"../html_data/{dataset}/treegen/v0715/{search_engine}html-{rewrite_method}-v0715-{dataset}-{split}.jsonl"
node_lines = [json.loads(line) for line in open(node_file)]



FileNotFoundError: [Errno 2] No such file or directory: '../html_data/hotpot-qa/treegen/v0715/binghtml-slimplmqr-v0715-hotpot-qa-test.jsonl'

In [45]:
from bs4 import BeautifulSoup
idx = 1
print(node_lines[idx].keys())
# dict_keys(['qa_pairs', 'wikipages', 'annotations', 'id', 'question', 'answer', 'short_answers', 'long_answers', 'html', 'paths', 'path_token_ids', 'node_tree', 'path_divisible'])
paths = node_lines[idx]['paths']
print(paths[0])
# ['html0', 'title']
node_tree = node_lines[idx]['node_tree']
print(node_tree[0])

html= node_lines[idx]['html']
soup = BeautifulSoup(html, 'html.parser')
parent=soup
for tag in paths[0]:
    for c in parent.contents:
        if c.name == tag:
            parent = c
            break
            
print(parent.get_text())

dict_keys(['id', 'output', 'question', 'answer', 'html', 'paths', 'path_token_ids', 'node_tree', 'path_divisible'])
['title']
-1|0.0|[]
l'Oiseau Blanc ("White Bird")


In [56]:
from transformers import AutoTokenizer

chat_tokenizer_path = "../../../huggingface/Meta-Llama-3.1-8B-Instruct"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_tokenizer_path, trust_remote_code=True)

In [64]:
from html4rag.html_utils import *
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

url="http://172.16.0.96/"
query_instruction_for_retrieval = "Represent this sentence for searching relevant passages: "


embedder = TEIEmbeddings(
            model=url,
            huggingfacehub_api_token="a-default-token",
            model_kwargs={"truncate": True})

nidx = 1
node_docs=[]
paths = node_lines[nidx]['paths']
html= node_lines[nidx]['html']
path_divisible = node_lines[nidx]['path_divisible']
question = query_instruction_for_retrieval + node_lines[nidx]['question']
soup = BeautifulSoup(html, 'html.parser')
for pidx in range(len(paths)):
    parent=soup
    for tag in paths[pidx]:
        for c in parent.contents:
            if c.name == tag:
                parent = c
                break
    node_docs.append(Document(page_content=parent.get_text(), metadata={"path_idx": pidx}))
batch_size = 16
db=FAISS.from_documents(node_docs[:batch_size], embedder)
for doc_batch_idx in range(batch_size, len(node_docs), batch_size):
    db.add_documents(node_docs[doc_batch_idx:doc_batch_idx+batch_size])

retriever = db.as_retriever(search_kwargs={"k": len(node_docs)})
ranked_docs=retriever.invoke(question)
path_rankings = [doc.metadata["path_idx"] for doc in ranked_docs]



In [65]:
from html4rag.html_utils import simplify_html, truncate_input, trim_path
#  trim html according to the path_rankings
import bs4
context_window = "2k"
max_context_window = int(context_window[:-1]) * 1000
             
        
paths = [{"path": paths[i], "divisible": path_divisible[i]} for i in range(len(paths))]   
for idj in range(len(paths)):
    path_idx = int(path_rankings[idj])
    paths[path_idx]["ranking"] = idj
    
soup = bs4.BeautifulSoup(html, 'html.parser')
for idj in range(len(paths)):
    path = paths[idj]["path"]
    tag = soup
    for p in path:
        for child in tag.contents:
            if isinstance(child, bs4.element.Tag):
                if child.name == p:
                    tag = child
                    break

    paths[idj]["tag"] = tag
    paths[idj]["token_length"] = len(chat_tokenizer.encode(str(tag), add_special_tokens=False))
    if paths[idj]["token_length"] < 64:
        #  move paths that are too short to the end
        paths[idj]["ranking"] = len(paths)
#  sort paths by ranking
paths = sorted(paths, key=lambda x: x["ranking"])
total_token_length = sum([p["token_length"] for p in paths])

#  remove low ranking paths
while total_token_length > max_context_window:
    if len(paths) == 1:
        break
    discarded_path = paths.pop()
    total_token_length -= discarded_path["token_length"]
    trim_path(discarded_path)

total_token_length = len(chat_tokenizer.encode(simplify_html(soup), add_special_tokens=False))
while total_token_length > max_context_window:
    if len(paths) == 1:
        break
    discarded_path = paths.pop()
    trim_path(discarded_path)
    total_token_length = len(chat_tokenizer.encode(simplify_html(soup), add_special_tokens=False))

if total_token_length > max_context_window:
    # loguru.logger.warning(f"dataset {dataset} sample {idx} cannot be trimmed to {max_context_window} tokens")
    html_trim = truncate_input(simplify_html(soup),chat_tokenizer, max_context_window)
else:
    html_trim = simplify_html(soup)

assert len(chat_tokenizer.encode(
    html_trim,
    add_special_tokens=False)) <= max_context_window, f"html length: {len(chat_tokenizer.encode(html_trim, add_special_tokens=False))}, max_context_window: {max_context_window}"

In [21]:
from tqdm import tqdm
import threading
#  chunk rerank and tree rerank
import json
import sys
sys.path.append("../")
from html4rag.html_utils import split_tree
from bs4 import BeautifulSoup

split = "test"
rewrite_method = "slimplmqr"
dataset = "hotpot-qa"
version="v0715"
search_engine="bing"
rerank_model = "bgelargeen"

context_window = "2k"
chat_tokenizer_name = "llama"
fine_trim_ratio = "1/2"
if fine_trim_ratio == "1/2":
    coarse_context_window = {"2k": "4k", "4k": "8k", "8k": "16k", "16k": "32k"}[context_window]
elif fine_trim_ratio == "2/3":
    coarse_context_window = {"2k": "3k", "4k": "6k", "8k": "12k", "16k": "24k"}[context_window]
else:
    raise ValueError(f"fine_trim_ratio {fine_trim_ratio} not supported")
data_file = f"../html_data/{dataset}/chunk-rerank/{chat_tokenizer_name}/{search_engine}html-{rewrite_method}-{rerank_model}-{dataset}-{split}-{coarse_context_window}.jsonl"
data_lines = [json.loads(line) for line in open(data_file)]
data_lines = data_lines[:10]
total_len=len(data_lines)

thread_pool = []

pbar = tqdm(total=total_len)
parallel_size = 8
def start_thread(rank):
    while len(data_lines) > 0:
        try:
            idx = total_len - len(data_lines)
            data_line = data_lines.pop(0)
            question = data_line['question']
            coarse_html_trim = data_lines[idx]["html_trim"]
        except Exception as e:
            pass
        pbar.update(1)


for i in range(parallel_size):
    thread = threading.Thread(target=start_thread, args=(i,))
    thread.start()
    thread_pool.append(thread)

for thread in thread_pool:
    thread.join()

pbar.close()



100%|██████████| 10/10 [00:00<00:00, 860.23it/s]


In [4]:
len(paths)

70

In [16]:
import os
for dataset in ["asqa", "hotpot-qa", "nq", "trivia-qa", "musique"]:
    dir_name=f"../html_data/{dataset}/treegen/"
    #  rename treegen to tree-gen
    os.rename(dir_name, dir_name.replace("treegen", "tree-gen"))

In [2]:
import requests
res=requests.post("http://bge-large-en.search.cls-3nbemh6i.ml.baichuan-inc.com/embed", json={"inputs":["就撒放结果飞机啊手工加工费。"*1000], "truncate":True})
res.text

'[[-0.009979298,0.010612546,-0.006892218,0.03191114,-0.016034726,-0.0029372496,-0.02347538,0.022321966,0.03374303,0.01972113,0.04566165,0.017606536,0.014971775,-0.063957974,-0.024334788,0.013965364,-0.009295166,-0.036637876,-0.026980856,0.007949514,-0.0064342446,-0.011121405,-0.04857911,-0.026257146,0.0054730657,0.038786396,0.010499466,0.02163218,0.06015849,0.0598871,-0.01393144,-0.015763333,0.01959674,-0.045186717,-0.025126347,-0.02487757,0.017572612,-0.009249933,-0.02797596,-0.031865906,0.008831537,-0.033607338,0.03593678,-0.043332204,-0.043716677,-0.013388656,-0.014892619,-0.069385804,0.020535303,-0.03926133,0.0023393398,-0.020648384,0.030418485,0.0073275752,0.02207319,0.009153815,-0.010776511,-0.023972932,-0.02623453,0.048172023,0.013004185,0.0081417505,0.021360787,-0.06431983,-0.012359629,0.030599412,0.010567314,-0.008927655,0.0028425453,-0.05848491,0.0038560235,0.015932953,0.0095495945,-0.039894577,0.020931084,0.0342632,-0.029762622,-0.0078760125,-0.03324548,0.014248064,0.0089955